In [ ]:
import pandas as pd
import numpy as np




## Load Data

In [ ]:
import os
# Data Paths
DATA_SUBFOLDER = 'playground-series-s4e8'
TRAIN_FILE_PATH = os.path.join(DATA_SUBFOLDER, "train.csv")
TEST_FILE_PATH = os.path.join(DATA_SUBFOLDER, "test.csv")

try:
    df_test_received  = pd.read_csv(TEST_FILE_PATH)
    df_train_received = pd.read_csv(TRAIN_FILE_PATH)
except FileNotFoundError:
    print("Error: train.csv or test.csv not found. Please ensure they are in the correct path.")
    raise # Stop execution if files are not found




## Data Split to make them < 100MB

###  Geting File Sizes

In [ ]:
colective_size = 95

# Getting Files Sizes
train_size_bytes = os.path.getsize(TRAIN_FILE_PATH)
test_size_bytes = os.path.getsize(TEST_FILE_PATH)

# Getting Row count
rows_in_train = len(df_train_received)
rows_in_test = len(df_test_received)

# MB Conversion
train_size_mb = train_size_bytes / (1024 * 1024)
test_size_mb = test_size_bytes / (1024 * 1024)

split_proporcion = train_size_mb/(test_size_mb + train_size_mb)

new_size_target_train  = split_proporcion * colective_size
new_size_target_test = colective_size - new_size_target_train

new_rows_ammount_train = int((new_size_target_train * rows_in_train)/train_size_mb)
new_rows_ammount_test = int((new_size_target_test * rows_in_test)/test_size_mb)



print(f"Rozmiar train.csv: {train_size_mb:.2f} MB rows {rows_in_train} new amount of lines { new_rows_ammount_train}")
print(f"Rozmiar test.csv: {test_size_mb:.2f} MB rows {rows_in_test} new amount of lines { new_rows_ammount_test}")




Wyliczenie proporcji wierszy by uzyskać 99 MB rozmiar pliku

In [ ]:
# Train target column
TARGET_COLUMN = 'class'

if TARGET_COLUMN in df_train_received.columns:
    print('TARGET_COLUMN found')
else:
    print('ERROR !!! TARGET_COLUMN not found !!!\n', df_train_received.columns)


df_train_reduced, _ = train_test_split(
    df_train_received,
    train_size=50000,
    stratify=df_train_received["target"],
    random_state=42
)
df_train_received: pełny DataFrame z wszystkimi kolumnami
train_size=50000: wybierasz 50 tys. wierszy
stratify=...: zachowujesz proporcje klas w kolumnie "target"
df_train_reduced: zawiera wszystkie kolumny, tylko mniej wierszy
_: pozostałe wiersze, których nie używasz

In [ ]:
NEW_DATA_SUBFOLDER = 'DATA_reduced_to_size_under_100_MB'
os.makedirs(NEW_DATA_SUBFOLDER, exist_ok=True)
NEW_TRAIN_FILE_PATH = os.path.join(NEW_DATA_SUBFOLDER, "train.csv")
NEW_TEST_FILE_PATH = os.path.join (NEW_DATA_SUBFOLDER, "test.csv")

from sklearn.model_selection import train_test_split

# Zmniejszamy zbiór do 50 tys. wierszy, zachowując proporcje targetu
df_train_reduced, _ = train_test_split(
    df_train_received,
    train_size= new_rows_ammount_train,
    stratify=df_train_received[TARGET_COLUMN],
    random_state=42
)

# Zapisz do nowego pliku
df_train_reduced.to_csv(NEW_TRAIN_FILE_PATH, index=False)

# Ustal liczbę wierszy , np. 30 tys. new_rows_ammount_test


# Losowe pobranie wierszy z testu
df_test_reduced = df_test_received.sample(n=new_rows_ammount_test, random_state=42)

# Zapisz do nowego pliku
df_test_reduced.to_csv(NEW_TEST_FILE_PATH, index=False)


# Getting Files Sizes
train_size_bytes = os.path.getsize(NEW_TRAIN_FILE_PATH)
test_size_bytes = os.path.getsize(NEW_TEST_FILE_PATH)

# Getting Row count
rows_in_train = len(df_train_received)
rows_in_test = len(df_test_received)

# MB Conversion
train_size_mb = train_size_bytes / (1024 * 1024)
test_size_mb = test_size_bytes / (1024 * 1024)

colective_size =train_size_mb + test_size_mb
print(f"Rozmiar train_reduced.csv: {train_size_mb:.2f} MB rows {rows_in_train}")
print(f"Rozmiar test_reduced.csv: {test_size_mb:.2f} MB rows {rows_in_test}")
print(f"Rozmiar SUM: {colective_size:.2f}")